In [ ]:
import pandas as pd
import xarray as xr
# import seaborn as sns
# from matplotlib.dates import DateFormatter
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
from glob import glob


In [ ]:
data = {}

In [ ]:
identifiers = ["20180101", "20180701", "20190101", "20190701"]

for identifier in identifiers:
    files = glob(f"*{identifier}*")
    csv_files = [file for file in files if file.endswith('.csv')]
    nc_files = [file for file in files if file.endswith('.nc')]

    if len(csv_files) + len(nc_files) == 3 and len(csv_files) == 2:
        data[identifier] = {
            "csv": csv_files,
            "nc": nc_files
        }


In [ ]:
def read_csv(path):
    data_finer = pd.read_csv(path)
    df_finer = pd.DataFrame({"time": pd.to_datetime(data_finer["date and time"])})
    df_finer["ZWL"] = data_finer["water level (m)"]
    return df_finer

def read_nc(path):
    data_coarser = xr.open_dataset(path)
    df_coarser = pd.DataFrame({'time': pd.to_datetime(data_coarser['time'].values)})
    df_coarser["ZWL"] = data_coarser["ZWL"].values
    return df_coarser

# df_coarser = read_nc("trih-coarser20180101.nc")
# df_finer1 = read_csv('trih-finer20180101_1-3water level.csv')
# df_finer2 = read_csv('trih-finer20180101_10-12water level.csv')

for identifier, file_dict in data.items():
    # print(file_dict)
    data[identifier]["df_finer"] = [read_csv(csv_file) for csv_file in file_dict["csv"]]
    data[identifier]["df_coarser"] = [read_nc(nc_file) for nc_file in file_dict["nc"]]
    assert len(data[identifier]["df_coarser"]) == 1, "There is more than one coarser file"
    assert len(data[identifier]["df_finer"]) == 2, "There is not two finer files"
    data[identifier]["df_coarser"] = data[identifier]["df_coarser"][0]


In [ ]:
def interpolate(small_df, large_df):
    # min_interval = '100T'
    min_interval = small_df['time'].diff().min()
    # print(min_interval)
    new_time_index_small = pd.date_range(start=small_df['time'].min(), end=small_df['time'].max(), freq=min_interval)
    new_time_index_large = pd.date_range(start=large_df['time'].min(), end=large_df['time'].max(), freq=min_interval)

    small_df_interpolated = small_df.set_index('time').reindex(new_time_index_small).interpolate()
    small_df_interpolated.index.name = 'time'
    small_df_interpolated.reset_index(inplace=True)

    large_df_interpolated = large_df.set_index('time').reindex(new_time_index_large).interpolate()
    large_df_interpolated.index.name = 'time'
    large_df_interpolated.reset_index(inplace=True)

    return small_df_interpolated, large_df_interpolated


# df_finer_interpolated = pd.concat([interpolate(df_finer1, df_coarser), interpolate(df_finer2, df_coarser)])

for identifier in data.keys():
    # data[identifier]["df_finer"] = []
    for i, df_finer in enumerate(data[identifier]["df_finer"]):
        df_finer_interpolated, df_coarser_interpolated = interpolate(df_finer, data[identifier]["df_coarser"])
        data[identifier]["df_finer"][i] = df_finer_interpolated
        data[identifier]["df_coarser"] = df_coarser_interpolated


In [ ]:
# sns.set(style="darkgrid")

# fig, ax = plt.subplots(figsize=(12, 4))

# ax.plot(df_finer_interpolated['time'], df_finer_interpolated['ZWL'], label='Finer', marker='o', color='red', markersize=1, linestyle='None')
# # ax.plot(df_finer_interpolated2['time'], df_finer_interpolated2['ZWL'], label='Finer2', marker='o', color='orange', markersize=1, linestyle='None')

# ax.plot(df_coarser['time'].values, df_coarser['ZWL'].values, label='BoB Model', color='blue')

# ax.set_xlabel('Time')
# ax.set_ylabel('Water Level (m)')
# ax.set_title('Water Level Over Time')

# ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
# plt.xticks(rotation=45)

# ax.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
def plot_water_level(df_finer_interpolated, df_coarser, identifier):
    fig = go.Figure()
    # print(len(df_finer_interpolated))
    print(identifier)

    fig.add_trace(go.Scatter(x=df_finer_interpolated[0]['time'], y=df_finer_interpolated[0]['ZWL'],
                            mode='markers', name='Finer', marker=dict(color='red', size=3)))
    fig.add_trace(go.Scatter(x=df_finer_interpolated[1]['time'], y=df_finer_interpolated[1]['ZWL'],
                            mode='markers', name='Finer', showlegend=False, marker=dict(color='red', size=3)))
    
    fig.add_trace(go.Scatter(x=df_coarser['time'], y=df_coarser['ZWL'],
                             mode='lines', name='BoB Model', line=dict(color='blue')))

    # Find the max ZWL in the finer data
    # max_zwl_idx1 = df_finer_interpolated[0]['ZWL'].idxmax() - 8
    # max_zwl_time1 = df_finer_interpolated[0]['time'].iloc[max_zwl_idx1]
    # max_zwl_value1 = df_finer_interpolated[0]['ZWL'].iloc[max_zwl_idx1]
    def find_max_time_zwl(df, index_offset, direction, minORmax):
        if minORmax == "max":
            if direction == "left":
                max_zwl_idx = df['ZWL'].idxmax() - index_offset
            elif direction == "right":
                max_zwl_idx = df['ZWL'].idxmax() + index_offset
            else:
                raise ValueError("Invalid direction")
        elif minORmax == "min":
            if direction == "left":
                max_zwl_idx = df['ZWL'].idxmin() - index_offset
            elif direction == "right":
                max_zwl_idx = df['ZWL'].idxmin() + index_offset
            else:
                raise ValueError("Invalid direction")
        
        max_zwl_time = df['time'].iloc[max_zwl_idx]
        first_time = df['time'].iloc[0]
        time_diff_minutes = (max_zwl_time - first_time).total_seconds() / 60
        rounded_diff_minutes = round(time_diff_minutes / 100) * 100
        # print(rounded_diff_minutes/100)
        max_zwl_time = first_time + pd.Timedelta(minutes=rounded_diff_minutes)
        max_zwl_idx = df.index[df['time'] == max_zwl_time][0]
        max_zwl_value = df['ZWL'].iloc[max_zwl_idx]
        return max_zwl_time, max_zwl_value, rounded_diff_minutes/100 + 1
    
    def find_min_time_zwl(df, index_offset, direction, max_zwl_idx):
        if direction == "left":
            min_zwl_idx = max_zwl_idx - index_offset
        elif direction == "right":
            min_zwl_idx = max_zwl_idx + index_offset
        else:
            raise ValueError("Invalid direction")
        
        min_zwl_time = df['time'].iloc[min_zwl_idx]
        first_time = df['time'].iloc[0]
        time_diff_minutes = (min_zwl_time - first_time).total_seconds() / 60
        rounded_diff_minutes = round(time_diff_minutes / 100) * 100
        min_zwl_time = first_time + pd.Timedelta(minutes=rounded_diff_minutes)
        min_zwl_idx = df.index[df['time'] == min_zwl_time][0]
        min_zwl_value = df['ZWL'].iloc[min_zwl_idx]
        return min_zwl_time, min_zwl_value

    max_zwl_time1, max_zwl_value1, idx_max1 = find_max_time_zwl(df_finer_interpolated[0], 8, "left", 'max')
    min_zwl_time1, min_zwl_value1, idx_min1 = find_max_time_zwl(df_finer_interpolated[0], 8, "left", 'min')

    max_zwl_time2, max_zwl_value2, idx_max2 = find_max_time_zwl(df_finer_interpolated[1], 8, "left", 'max')
    min_zwl_time2, min_zwl_value2, idx_min2 = find_max_time_zwl(df_finer_interpolated[1], 8, "left", 'min')

    fig.add_trace(go.Scatter(x=[max_zwl_time1], y=[max_zwl_value1],
                             mode='markers+text', showlegend=False, marker=dict(color='red', size=10),
                             text=str(max_zwl_time1)+ "---" + str(idx_max1), textposition='top center'))
    
    fig.add_trace(go.Scatter(x=[min_zwl_time1], y=[min_zwl_value1],
                             mode='markers+text', showlegend=False, marker=dict(color='red', size=10),
                             text=str(min_zwl_time1) + "---" + str(idx_min1), textposition='top center'))


    fig.add_trace(go.Scatter(x=[max_zwl_time2], y=[max_zwl_value2],
                             mode='markers+text', showlegend=False, marker=dict(color='red', size=10),
                             text=str(max_zwl_time2) + "---" + str(idx_max2), textposition='top center'))


    fig.add_trace(go.Scatter(x=[min_zwl_time2], y=[min_zwl_value2],
                             mode='markers+text', showlegend=False, marker=dict(color='red', size=10),
                             text=str(min_zwl_time2) + "---" + str(idx_min2), textposition='top center'))




    fig.update_layout(
        title='Water Level Over Time',
        xaxis_title='Time',
        yaxis_title='Water Level (m)',
        # xaxis=dict(
        #     tickformat='%Y-%m-%d',
        #     tickangle=45
        # ),
        plot_bgcolor='lightblue',
        paper_bgcolor='lightyellow',
        xaxis=dict(gridcolor='white'),
        yaxis=dict(gridcolor='white'),
        # template='plotly_white'
    )

    fig.update_layout(width=1000)
    fig.show()
    fig.write_image(f"Nested_{identifier}.png")

# plot_water_level(data[identifiers[0]]["df_finer"], data[identifiers[0]]["df_coarser"], identifiers[0])
for identifier in data.keys():
    plot_water_level(data[identifier]["df_finer"], data[identifier]["df_coarser"], identifier)
    # break

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# # from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
# # from matplotlib.figure import Figure
# # fig, ax = plt.subplots(2, 2,figsize=(12, 6))

# def plot_water_level(df_finer_interpolated, df_coarser, identifier, ax):
#     # fig, ax = plt.subplots(figsize=(12, 6))
#     # print(identifier)

#     # Plot finer data
#     ax.scatter(df_finer_interpolated[0]['time'], df_finer_interpolated[0]['ZWL'], 
#                color='red', s=3, label='Finer')
#     ax.scatter(df_finer_interpolated[1]['time'], df_finer_interpolated[1]['ZWL'], 
#                color='red', s=3)
    
#     # Plot coarser data
#     ax.plot(df_coarser['time'], df_coarser['ZWL'], color='blue', label='BoB Model')

#     def find_max_time_zwl(df, index_offset, direction, minORmax):
#         if minORmax == "max":
#             if direction == "left":
#                 max_zwl_idx = df['ZWL'].idxmax() - index_offset
#             elif direction == "right":
#                 max_zwl_idx = df['ZWL'].idxmax() + index_offset
#             else:
#                 raise ValueError("Invalid direction")
#         elif minORmax == "min":
#             if direction == "left":
#                 max_zwl_idx = df['ZWL'].idxmin() - index_offset
#             elif direction == "right":
#                 max_zwl_idx = df['ZWL'].idxmin() + index_offset
#             else:
#                 raise ValueError("Invalid direction")
        
#         max_zwl_time = df['time'].iloc[max_zwl_idx]
#         first_time = df['time'].iloc[0]
#         time_diff_minutes = (max_zwl_time - first_time).total_seconds() / 60
#         rounded_diff_minutes = round(time_diff_minutes / 100) * 100
#         max_zwl_time = first_time + pd.Timedelta(minutes=rounded_diff_minutes)
#         max_zwl_idx = df.index[df['time'] == max_zwl_time][0]
#         max_zwl_value = df['ZWL'].iloc[max_zwl_idx]
#         return max_zwl_time, max_zwl_value, rounded_diff_minutes/100 + 1
    
#     max_zwl_time1, max_zwl_value1, idx_max1 = find_max_time_zwl(df_finer_interpolated[0], 8, "left", 'max')
#     min_zwl_time1, min_zwl_value1, idx_min1 = find_max_time_zwl(df_finer_interpolated[0], 8, "left", 'min')

#     max_zwl_time2, max_zwl_value2, idx_max2 = find_max_time_zwl(df_finer_interpolated[1], 8, "left", 'max')
#     min_zwl_time2, min_zwl_value2, idx_min2 = find_max_time_zwl(df_finer_interpolated[1], 8, "left", 'min')

#     # Plot max and min points
#     ax.scatter([max_zwl_time1], [max_zwl_value1], color='red', s=50)
#     ax.annotate(f"{max_zwl_time1}---{idx_max1}", (max_zwl_time1, max_zwl_value1), 
#                 xytext=(0, 10), textcoords='offset points', ha='center')

#     ax.scatter([min_zwl_time1], [min_zwl_value1], color='red', s=50)
#     ax.annotate(f"{min_zwl_time1}---{idx_min1}", (min_zwl_time1, min_zwl_value1), 
#                 xytext=(0, 10), textcoords='offset points', ha='center')

#     ax.scatter([max_zwl_time2], [max_zwl_value2], color='red', s=50)
#     ax.annotate(f"{max_zwl_time2}---{idx_max2}", (max_zwl_time2, max_zwl_value2), 
#                 xytext=(0, 10), textcoords='offset points', ha='center')

#     ax.scatter([min_zwl_time2], [min_zwl_value2], color='red', s=50)
#     ax.annotate(f"{min_zwl_time2}---{idx_min2}", (min_zwl_time2, min_zwl_value2), 
#                 xytext=(0, 10), textcoords='offset points', ha='center')

#     # Set labels and title
#     ax.set_xlabel('Time')
#     ax.set_ylabel('Water Level (m)')
#     ax.set_title('Water Level Over Time')

#     # Format x-axis
#     ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
#     plt.xticks(rotation=45)

#     # Add legend
#     ax.legend()

#     # Adjust layout
#     # plt.tight_layout()

#     # Show plot
#     # plt.show()

#     # Save plot
#     # plt.savefig(f"water_level_plot_{identifier}.png")
#     # plt.close()

# fig, axs = plt.subplots(2, 2, figsize=(15, 10))
# # plot_water_level(data[identifiers[0]]["df_finer"], data[identifiers[0]]["df_coarser"], identifiers[0])
# for identifier, ax in zip(data.keys(), axs.flatten()):
#     plt.sca(ax)
#     plot_water_level(data[identifier]["df_finer"], data[identifier]["df_coarser"], identifier, ax)
#     # break
